In [ ]:
# coding: utf-8
from Crypto.Cipher import AES
import base64
import requests
import sys

headers = {
    'Cookie': 'appver=1.5.0.75771;',
    'Referer': 'http://music.163.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
}

# first_param = "{rid:\"\", offset:\"0\", total:\"true\", limit:\"20\", csrf_token:\"\"}"
first_param = "{\"ids\":\"[%d]\",\"br\":128000,\"csrf_token\":\"\"}"
second_param = "010001"
third_param = "00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7"
forth_param = "0CoJUm6Qyw8W8jud"


def get_params():
    iv = "0102030405060708"
    first_key = forth_param
    second_key = 16 * 'F'
    h_encText = AES_encrypt(first_param, first_key, iv)
    h_encText = AES_encrypt(h_encText, second_key, iv)
    return h_encText


def get_encSecKey():
    encSecKey = "257348aecb5e556c066de214e531faadd1c55d814f9be95fd06d6bff9f4c7a41f831f6394d5a3fd2e3881736d94a02ca919d952872e7d0a50ebfa1769a7a62d512f5f1ca21aec60bc3819a9c3ffca5eca9a0dba6d6f7249b06f5965ecfff3695b54e1c28f3f624750ed39e7de08fc8493242e26dbc4484a01c76f739e135637c"
    return encSecKey


def AES_encrypt(text, key, iv):
    pad = 16 - len(text) % 16
    if isinstance(text, str):
        text = text + pad * chr(pad)
    else:
        text = text.decode('utf-8') + pad * chr(pad)
    encryptor = AES.new(key, AES.MODE_CBC, iv)
    encrypt_text = encryptor.encrypt(text)
    encrypt_text = base64.b64encode(encrypt_text)
    return encrypt_text


def get_json(url, params, encSecKey):
    data = {
        "params": params,
        "encSecKey": encSecKey
    }
    response = requests.post(url, headers=headers, data=data).json()
    return response['data']


music_id = input('请输入歌曲ID：')
first_param = "{\"ids\":\"[%d]\",\"br\":128000,\"csrf_token\":\"\"}" % int(music_id)
url = 'https://music.163.com/weapi/song/enhance/player/url?csrf_token='
params = get_params()
encSecKey = get_encSecKey()
"""
rsp:{
    'data': [{
        'gain': 2.3073,
        'type': 'mp3',
        'url': 'http://m10.music.126.net/20180111133509/24c79548414f7aa7407985818cb16a39/ymusic/333c/66b1/e5ec/
            72aeb13aca24c989295e58e8384e3f97.mp3',
        'md5': '72aeb13aca24c989295e58e8384e3f97',
        'flag': 0, 'code': 200, 'payed': 0, 'id': 151619, 'expi': 1200, 'size': 3868307,
        'uf': None, 'br': 128000, 'fee': 0, 'canExtend': False}], 'code': 200}
"""
rsp = get_json(url, params, encSecKey)
music_url = rsp[0].get('url')
if music_url:
    music = requests.get(music_url)
    name = sys.path[0] + "/%s.mp3" % rsp[0]['id']
    with open(name, "wb") as code:
        code.write(music.content)

In [18]:
# -*- coding: UTF-8 –*-
'''
网易云音乐爬虫，现可以进行评论（热门评论以及全部评论）的爬取
作者：洪韬
时间：2020/3/11
'''
from Crypto.Cipher import AES
import base64
import requests
import json
import time
import csv
import os
import re



class wangyiCrawl:
    def __init__(self):
        self.Headers = {
            'Accept': "*/*",
            'Accept-Language': "zh-CN,zh;q=0.9",
            'Connection': "keep-alive",
            'Host': "music.163.com",
            'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.79 Safari/537.36"

        }

        # 第二个参数
        self.second_param = "010001"
        # 第三个参数
        self.third_param = "00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7"
        # 第四个参数
        self.forth_param = "0CoJUm6Qyw8W8jud"

    def __create_path(self, *path):
        finPath = ''
        for i in range(len(path)):
            finPath += path[i]
            if not os.path.exists(finPath):
                os.mkdir(finPath)
            finPath += '/'

    def __get_params(self, page):
        # 获取encText，也就是params
        iv = "0102030405060708"
        first_key = self.forth_param
        second_key = 'F' * 16
        if page == 0:
            first_param = '{rid:"", offset:"0", total:"true", limit:"20", csrf_token:""}'
        else:
            offset = str((page - 1) * 20)
            first_param = '{rid:"", offset:"%s", total:"%s", limit:"20", csrf_token:""}' % (offset, 'false')
        self.encText = self.__AES_encrypt(first_param, first_key, iv)
        self.encText = self.__AES_encrypt(self.encText.decode('utf-8'), second_key, iv)
        return self.encText

    def __AES_encrypt(self, text, key, iv):
        # AES加密
        pad = 16 - len(text) % 16
        text = text + pad * chr(pad)
        encryptor = AES.new(key.encode('utf-8'), AES.MODE_CBC, iv.encode('utf-8'))
        encrypt_text = encryptor.encrypt(text.encode('utf-8'))
        encrypt_text = base64.b64encode(encrypt_text)
        return encrypt_text

    # 获取热评以及全部评论
    def get_comments(self, url, path):
        rsp = requests.get(url.replace('#', ''), headers=self.Headers)
        title = re.compile('"title": "(.*?)",').findall(rsp.text)[0]
        self.__create_path(path, title)
        file1 = open(path + '/' + title + '/hotComments.csv', 'w', encoding='utf-8', newline='')
        file2 = open(path + '/' + title + '/comments.csv', 'w', encoding='utf-8', newline='')
        fieldnames = ['user', 'thumbs-up', 'time', 'comments']
        writer1 = csv.DictWriter(file1, fieldnames=fieldnames)
        writer1.writeheader()
        writer2 = csv.DictWriter(file2, fieldnames=fieldnames)
        writer2.writeheader()
        id = url.replace('https://music.163.com/#/song?id=', '')
        url = 'https://music.163.com/weapi/v1/resource/comments/R_SO_4_' + id + '?csrf_token'
        params = self.__get_params(1)
        encSecKey = "257348aecb5e556c066de214e531faadd1c55d814f9be95fd06d6bff9f4c7a41f831f6394d5a3fd2e3881736d94a02ca919d952872e7d0a50ebfa1769a7a62d512f5f1ca21aec60bc3819a9c3ffca5eca9a0dba6d6f7249b06f5965ecfff3695b54e1c28f3f624750ed39e7de08fc8493242e26dbc4484a01c76f739e135637c"
        self.post = {
            'params': params,
            'encSecKey': encSecKey,
        }
        response = requests.post(url, headers=self.Headers, data=self.post)
        json_dict = json.loads(response.text)
        hotcomments = json_dict['hotComments']
        for i in hotcomments:
            time_local = time.localtime(int(i['time'] / 1000))  # 将毫秒级时间转换为日期
            dt = time.strftime("%Y-%m-%d %H:%M:%S", time_local)
            itemDict = {
                'user': i['user']['nickname'],
                'thumbs-up': str(i['likedCount']),
                'time': dt,
                'comments': i['content'].replace('\n', '')
            }
            writer1.writerow(itemDict)
        file1.close()
        comments_num = int(json_dict['total'])
        present_page = 0
        if (comments_num % 20 == 0):
            page = comments_num / 20
        else:
            page = int(comments_num / 20) + 1
        print("共有%d页评论" % page)
        # 逐页抓取
        for i in range(int(page)):
            print('正在爬取第' + str(i + 1) + '页')
            params = self.__get_params(i + 1)
            encSecKey = encSecKey
            self.post = {
                'params': params,
                'encSecKey': encSecKey,
            }
            response = requests.post(url, headers=self.Headers, data=self.post)
            json_dict = json.loads(response.text)
            present_page = present_page + 1
            for i in json_dict['comments']:
                time_local = time.localtime(int(i['time'] / 1000))  # 将毫秒级时间转换为日期
                dt = time.strftime("%Y-%m-%d %H:%M:%S", time_local)
                itemDict = {
                    'user': i['user']['nickname'],
                    'thumbs-up': str(i['likedCount']),
                    'time': dt,
                    'comments': i['content'].replace('\n', '')
                }
                writer2.writerow(itemDict)
        file2.close()


if __name__ == '__main__':
    app = wangyiCrawl()
    #获取网易云音乐热门评论以及大概101页的评论
    app.get_comments('https://music.163.com/#/song?id=254574','data')

共有15074页评论
正在爬取第1页
正在爬取第2页
正在爬取第3页
正在爬取第4页
正在爬取第5页
正在爬取第6页
正在爬取第7页
正在爬取第8页
正在爬取第9页
正在爬取第10页
正在爬取第11页
正在爬取第12页
正在爬取第13页
正在爬取第14页
正在爬取第15页
正在爬取第16页
正在爬取第17页
正在爬取第18页
正在爬取第19页
正在爬取第20页
正在爬取第21页
正在爬取第22页
正在爬取第23页
正在爬取第24页
正在爬取第25页
正在爬取第26页
正在爬取第27页
正在爬取第28页
正在爬取第29页
正在爬取第30页
正在爬取第31页
正在爬取第32页
正在爬取第33页
正在爬取第34页
正在爬取第35页
正在爬取第36页
正在爬取第37页
正在爬取第38页
正在爬取第39页
正在爬取第40页
正在爬取第41页
正在爬取第42页
正在爬取第43页
正在爬取第44页
正在爬取第45页
正在爬取第46页
正在爬取第47页
正在爬取第48页
正在爬取第49页
正在爬取第50页
正在爬取第51页
正在爬取第52页
正在爬取第53页
正在爬取第54页
正在爬取第55页
正在爬取第56页
正在爬取第57页
正在爬取第58页
正在爬取第59页
正在爬取第60页
正在爬取第61页
正在爬取第62页
正在爬取第63页
正在爬取第64页
正在爬取第65页
正在爬取第66页
正在爬取第67页
正在爬取第68页
正在爬取第69页
正在爬取第70页
正在爬取第71页
正在爬取第72页
正在爬取第73页
正在爬取第74页
正在爬取第75页
正在爬取第76页
正在爬取第77页
正在爬取第78页
正在爬取第79页
正在爬取第80页
正在爬取第81页
正在爬取第82页
正在爬取第83页
正在爬取第84页
正在爬取第85页
正在爬取第86页
正在爬取第87页
正在爬取第88页
正在爬取第89页
正在爬取第90页
正在爬取第91页
正在爬取第92页
正在爬取第93页
正在爬取第94页
正在爬取第95页
正在爬取第96页
正在爬取第97页
正在爬取第98页
正在爬取第99页
正在爬取第100页
正在爬取第101页
正在爬取第102页
正在爬取第103页
正在爬取第104页
正在爬取第105页
正在爬取第106页
正在爬取第107页
正在爬取第108页
正在爬取第109页
正在爬取第11

正在爬取第830页
正在爬取第831页
正在爬取第832页
正在爬取第833页
正在爬取第834页
正在爬取第835页
正在爬取第836页
正在爬取第837页
正在爬取第838页
正在爬取第839页
正在爬取第840页
正在爬取第841页
正在爬取第842页
正在爬取第843页
正在爬取第844页
正在爬取第845页
正在爬取第846页
正在爬取第847页
正在爬取第848页
正在爬取第849页
正在爬取第850页
正在爬取第851页
正在爬取第852页
正在爬取第853页
正在爬取第854页
正在爬取第855页
正在爬取第856页
正在爬取第857页
正在爬取第858页
正在爬取第859页
正在爬取第860页
正在爬取第861页
正在爬取第862页
正在爬取第863页
正在爬取第864页
正在爬取第865页
正在爬取第866页
正在爬取第867页
正在爬取第868页
正在爬取第869页
正在爬取第870页
正在爬取第871页
正在爬取第872页
正在爬取第873页
正在爬取第874页
正在爬取第875页
正在爬取第876页
正在爬取第877页
正在爬取第878页
正在爬取第879页
正在爬取第880页
正在爬取第881页
正在爬取第882页
正在爬取第883页
正在爬取第884页
正在爬取第885页
正在爬取第886页
正在爬取第887页
正在爬取第888页
正在爬取第889页
正在爬取第890页
正在爬取第891页
正在爬取第892页
正在爬取第893页
正在爬取第894页
正在爬取第895页
正在爬取第896页
正在爬取第897页
正在爬取第898页
正在爬取第899页
正在爬取第900页
正在爬取第901页
正在爬取第902页
正在爬取第903页
正在爬取第904页
正在爬取第905页
正在爬取第906页
正在爬取第907页
正在爬取第908页
正在爬取第909页
正在爬取第910页
正在爬取第911页
正在爬取第912页
正在爬取第913页
正在爬取第914页
正在爬取第915页
正在爬取第916页
正在爬取第917页
正在爬取第918页
正在爬取第919页
正在爬取第920页
正在爬取第921页
正在爬取第922页
正在爬取第923页
正在爬取第924页
正在爬取第925页
正在爬取第926页
正在爬取第927页
正在爬取第928页
正在爬取第929页


正在爬取第1591页
正在爬取第1592页
正在爬取第1593页
正在爬取第1594页
正在爬取第1595页
正在爬取第1596页
正在爬取第1597页
正在爬取第1598页
正在爬取第1599页
正在爬取第1600页
正在爬取第1601页
正在爬取第1602页
正在爬取第1603页
正在爬取第1604页
正在爬取第1605页
正在爬取第1606页
正在爬取第1607页
正在爬取第1608页
正在爬取第1609页
正在爬取第1610页
正在爬取第1611页
正在爬取第1612页
正在爬取第1613页
正在爬取第1614页
正在爬取第1615页
正在爬取第1616页
正在爬取第1617页
正在爬取第1618页
正在爬取第1619页
正在爬取第1620页
正在爬取第1621页
正在爬取第1622页
正在爬取第1623页
正在爬取第1624页
正在爬取第1625页
正在爬取第1626页
正在爬取第1627页
正在爬取第1628页
正在爬取第1629页
正在爬取第1630页
正在爬取第1631页
正在爬取第1632页
正在爬取第1633页
正在爬取第1634页
正在爬取第1635页
正在爬取第1636页
正在爬取第1637页
正在爬取第1638页
正在爬取第1639页
正在爬取第1640页
正在爬取第1641页
正在爬取第1642页
正在爬取第1643页
正在爬取第1644页
正在爬取第1645页
正在爬取第1646页
正在爬取第1647页
正在爬取第1648页
正在爬取第1649页
正在爬取第1650页
正在爬取第1651页
正在爬取第1652页
正在爬取第1653页
正在爬取第1654页
正在爬取第1655页
正在爬取第1656页
正在爬取第1657页
正在爬取第1658页
正在爬取第1659页
正在爬取第1660页
正在爬取第1661页
正在爬取第1662页
正在爬取第1663页
正在爬取第1664页
正在爬取第1665页
正在爬取第1666页
正在爬取第1667页
正在爬取第1668页
正在爬取第1669页
正在爬取第1670页
正在爬取第1671页
正在爬取第1672页
正在爬取第1673页
正在爬取第1674页
正在爬取第1675页
正在爬取第1676页
正在爬取第1677页
正在爬取第1678页
正在爬取第1679页
正在爬取第1680页
正在爬取第1681页

ConnectionError: HTTPSConnectionPool(host='music.163.com', port=443): Max retries exceeded with url: /weapi/v1/resource/comments/R_SO_4_254574?csrf_token (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x120a32358>: Failed to establish a new connection: [Errno 61] Connection refused'))